### Achived Statements - indexing

Target:

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 3,239,979,516
 
Process:

 - bulk index the 3B archived statements using BAF
 
#### Principles

 - scale out the number of Nuxeo Worker nodes 
     - start with 5 worker nodes
     - ready to scale up to 8
     - bulk reindexing is configured with
         - 40 partitions
         - 5 threads/node to creating indexing commands
         - 4 threads/node to submit indexing commands


    es.baf.index.defaultPartitions=40
    es.baf.index.concurrency=5
    es.baf.bulkIndex.concurrency=4


- configure ES for bulk indexing (see [ES toolbox](../ToolBox/Elasticsearch.ipynb))
     - no replicas
     - refresh rate  
 - configure the archives MongoDB cluster use allow read from secondaries
     - add `?readPreference=nearest` to the connection url




Associated Datadog notebook: https://app.datadoghq.com/notebook/276275/index-archives-3b

#### Start the Bulk Indexing

In [11]:
!(INJECTOR="http://127.0.0.1:8080";\
curl -H 'Content-Type:application/json+nxrequest' \
  -H 'X-NXRepository:archives' \
  -X POST -d '{"params":{},"context":{}}' -u $NXUSER:$NXPWD \
  "$INJECTOR/nuxeo/api/v1/automation/Elasticsearch.BulkIndex")

{"commandId":"7847045a-298f-4341-959e-f6b8dc971ee0"}

In [12]:
# check status
! cid="7847045a-298f-4341-959e-f6b8dc971ee0"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

{"entity-type":"bulkStatus","commandId":"7847045a-298f-4341-959e-f6b8dc971ee0","state":"SCROLLING_RUNNING","processed":281000,"error":false,"errorCount":0,"total":0,"action":"index","username":"nco-admin","submitted":"2020-09-19T14:34:19.857Z","scrollStart":"2020-09-19T14:34:19.881Z","scrollEnd":null,"processingStart":null,"processingEnd":null,"completed":null,"processingMillis":0}

#### Configure index for maximum speed

In [13]:
Index = "archives"

cmd = """
curl -X PUT $ES_SERVER/$Index/_settings -H "Content-Type: application/json" \
   -d '{"index" : { "number_of_replicas" : 0  , "refresh_interval" : -1 } }'

""".replace("$Index", Index)

print(cmd)

!$cmd


curl -X PUT $ES_SERVER/archives/_settings -H "Content-Type: application/json"    -d '{"index" : { "number_of_replicas" : 0  , "refresh_interval" : -1 } }'


{"acknowledged":true}

Get Bulk Command Status

In [14]:
# check status
! cid="7847045a-298f-4341-959e-f6b8dc971ee0"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

{"entity-type":"bulkStatus","commandId":"7847045a-298f-4341-959e-f6b8dc971ee0","state":"RUNNING","processed":2639789000,"error":false,"errorCount":0,"total":3240123297,"action":"index","username":"nco-admin","submitted":"2020-09-19T14:34:19.857Z","scrollStart":"2020-09-19T14:34:19.881Z","scrollEnd":"2020-09-19T22:17:36.630Z","processingStart":null,"processingEnd":null,"completed":null,"processingMillis":0}

**Gather statistics** 

In [ ]:
!statfile="snapshots/es-3B-archive-indexing-"`date +%Y%m%d-%H%M%S`.txt; \
 curl "$ES_SERVER" > $statfile; \
 curl "$ES_SERVER/_cat/health?v" >> $statfile ; \
 curl "$ES_SERVER/_cat/nodes?v" >> $statfile; \
 curl "$ES_SERVER/_cat/indices?v" >> $statfile; \
 cat $statfile